Insert the transcript file 


In [9]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq groq==0.13.0 --progress-bar off
!pip install -qqq python-dotenv==1.0.1 --progress-bar off
!pip install groq

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
file_path = 'adg0008_er_2024_10_31.csv'
input_data = pd.read_csv(file_path, sep=None, engine='python')


if 'Transkript' in input_data.columns:
    transcript_data = input_data[['Transkript']].dropna().reset_index(drop=True)
    print(transcript_data.head())
else:
    print("The 'Transkript' column was not found in the file.")

                                          Transkript
0                                    [Bandgeräusche]
1  Also, ich erzähl Ihnen das so zunächst wie mir...
2                    [unterbricht] Wie Ihnen das ...
3  Ich habe mich ja für nichts vorbereitet, so ir...
4                   Genau.[Bandgeräusche] [x][Pause]


In [3]:
combined_transcript = " ".join(transcript_data['Transkript'])
print(combined_transcript)

[Bandgeräusche] Also, ich erzähl Ihnen das so zunächst wie mir der Mund ... [unterbricht] Wie Ihnen das ... Ich habe mich ja für nichts vorbereitet, so irgendwie ... Genau.[Bandgeräusche] [x][Pause] Ich bin also eingeborener Altendorfer [Stadtteil von Essen]. Ich bin geboren in dieser Gemeinde, und zwar in einem Haus in der Geißlerstraße, Ecke Röntgenstraße, wo später die Gastwirtschaft Klute drin gewesen ist. Die ist ungefähr diagonal von der Christuskirche, wenn Sie die hier kennen. Hm. Ja. Wir haben dann in der Heilsmannstraße 28 gewohnt, bis 1917. Das waren Häuser von einem Hausbesitzer (Rüken ?), ich weiß nicht, was aus dem geworden ist. Könnte ich mich auch, wenn ich ..., wenn das irgendwie interessiert, dafür interessieren ... Ich weiß noch jemand, der die (Einjährigen ?) kennt ... Und haben wir in der Markscheide 17, das ist genau gegenüber der Heintzmannstraße, auf der anderen Seite, Altendorfer Straße, gewohnt. Hm. Und zwar bis 1927. Und dann 1927, nachdem wir schon längere Z

In [4]:
# Function to split transcript into chunks based on model's context length
def split_transcript_for_model(transcript, max_tokens=2048, token_ratio=1.3):
    max_words = int(max_tokens / token_ratio)
    words = transcript.split()
    chunks = [" ".join(words[i:i+max_words]) for i in range(0, len(words), max_words)]
    return chunks

# Set context length and token ratio
context_length = 2048  # Adjust to 4096 if using extended context models
token_to_word_ratio = 1.3

# Split the combined transcript into chunks
chunks = split_transcript_for_model(combined_transcript, max_tokens=context_length, token_ratio=token_to_word_ratio)

# Display the number of chunks and the first chunk
print(f"Number of chunks: {len(chunks)}")
print(chunks[0])


Number of chunks: 49
[Bandgeräusche] Also, ich erzähl Ihnen das so zunächst wie mir der Mund ... [unterbricht] Wie Ihnen das ... Ich habe mich ja für nichts vorbereitet, so irgendwie ... Genau.[Bandgeräusche] [x][Pause] Ich bin also eingeborener Altendorfer [Stadtteil von Essen]. Ich bin geboren in dieser Gemeinde, und zwar in einem Haus in der Geißlerstraße, Ecke Röntgenstraße, wo später die Gastwirtschaft Klute drin gewesen ist. Die ist ungefähr diagonal von der Christuskirche, wenn Sie die hier kennen. Hm. Ja. Wir haben dann in der Heilsmannstraße 28 gewohnt, bis 1917. Das waren Häuser von einem Hausbesitzer (Rüken ?), ich weiß nicht, was aus dem geworden ist. Könnte ich mich auch, wenn ich ..., wenn das irgendwie interessiert, dafür interessieren ... Ich weiß noch jemand, der die (Einjährigen ?) kennt ... Und haben wir in der Markscheide 17, das ist genau gegenüber der Heintzmannstraße, auf der anderen Seite, Altendorfer Straße, gewohnt. Hm. Und zwar bis 1927. Und dann 1927, nachde

In [5]:
# import re

# def clean_transcript(text):
#     # Remove non-verbal annotations (e.g., [Bandgeräusche], [Pause])
#     text = re.sub(r"\[.*?\]", "", text)
    
#     # Remove extra spaces and leading/trailing whitespace
#     text = re.sub(r"\s+", " ", text).strip()
    
#     return text

# # Apply cleaning to each chunk
# cleaned_chunks = [clean_transcript(chunk) for chunk in chunks]

# # Display the first cleaned chunk
# print(cleaned_chunks[0])

Also, ich erzähl Ihnen das so zunächst wie mir der Mund ... Wie Ihnen das ... Ich habe mich ja für nichts vorbereitet, so irgendwie ... Genau. Ich bin also eingeborener Altendorfer . Ich bin geboren in dieser Gemeinde, und zwar in einem Haus in der Geißlerstraße, Ecke Röntgenstraße, wo später die Gastwirtschaft Klute drin gewesen ist. Die ist ungefähr diagonal von der Christuskirche, wenn Sie die hier kennen. Hm. Ja. Wir haben dann in der Heilsmannstraße 28 gewohnt, bis 1917. Das waren Häuser von einem Hausbesitzer (Rüken ?), ich weiß nicht, was aus dem geworden ist. Könnte ich mich auch, wenn ich ..., wenn das irgendwie interessiert, dafür interessieren ... Ich weiß noch jemand, der die (Einjährigen ?) kennt ... Und haben wir in der Markscheide 17, das ist genau gegenüber der Heintzmannstraße, auf der anderen Seite, Altendorfer Straße, gewohnt. Hm. Und zwar bis 1927. Und dann 1927, nachdem wir schon längere Zeit Mitglied dieser Genossenschaft, die hieß früher Kleinhaussiedlung, gewese

In [6]:
# Structure the cleaned chunks into a DataFrame
structured_data = pd.DataFrame({
    "Chunk_ID": range(1, len(cleaned_chunks) + 1),
    "Transcript_Text": cleaned_chunks
})

# Display the structured data
print(structured_data.head())

   Chunk_ID                                    Transcript_Text
0         1  Also, ich erzähl Ihnen das so zunächst wie mir...
1         2  Krupp gehabt habe, bekommen. Da war ein Mann n...
2         3  Weil, der ganze Waschprozess in der Küche und ...
3         4  Material, was sie raus brachten, gemeinschaftl...
4         5  Aber wir hatten noch Eier. Ja, ja, genau. Und ...


In [7]:
from groq import Groq
import pandas as pd 

# Directly set the API key
api_key = "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" 

# Initialize the Groq client
client = Groq(api_key=api_key)

print("Groq client initialized successfully!")

Groq client initialized successfully!


In [8]:
import pandas as pd

def extract_metadata_from_chunks(client, model_name: str, chunks: list) -> pd.DataFrame:
    """
    Extract metadata from multiple chunks of a German transcript using the specified model.
    
    Args:
        client: The Groq client object.
        model_name: The name of the model to use (e.g., 'llama3-8b-8192').
        chunks: A list of transcript chunks to analyze.
    
    Returns:
        A DataFrame containing the extracted metadata for all chunks.
    """
    all_metadata = []

    for i, chunk in enumerate(chunks):
        print(f"Processing chunk {i+1}/{len(chunks)}...")
        
        # Prompt for metadata extraction
        prompt = f"""
        Extrahieren Sie die folgenden Informationen aus dem Transkript und geben Sie die Antwort auf Deutsch:

        ### Persönliche Informationen:
        1. NAME: Der vollständige Name der Person (Vor- und Nachname, falls verfügbar).
        2. JAHRGANG: Das Geburtsjahr der Person.
        3. ORT: Der Geburtsort der Person.
        4. GESCHLECHT: Das Geschlecht der Person (z.B. männlich, weiblich, divers).
        5. BERUF: Der aktuelle Beruf der Person.

        ### Informationen über Vater und Partner:
        6. VAT_JG: Geburtsjahr des Vaters.
        7. VAT_KONFESSION: Religion des Vaters.
        8. VAT_HERKUN: Herkunft des Vaters.
        9. VAT_SCHULE: Schulbildung des Vaters.
        10. VAT_AUSBIL: Ausbildung des Vaters.
        11. VAT_STAND: Beruflicher Status des Vaters.
        12. VAT_POLOR: Politische Orientierung des Vaters.

        Transkript:
        {chunk}

        Metadaten:
        NAME:
        JAHRGANG:
        ORT:
        GESCHLECHT:
        BERUF:
        VAT_JG:
        VAT_KONFESSION:
        VAT_HERKUN:
        VAT_SCHULE:
        VAT_AUSBIL:
        VAT_STAND:
        VAT_POLOR:
        """
        
        # Generate response using the Groq client
        response = client.chat.completions.create(
            model=model_name,
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
        )
        
        # Extract metadata from the response
        metadata = response.choices[0].message.content

        # Parse metadata into a DataFrame
        numbered_lines = [
            line for line in metadata.split("\n") 
            if line.strip().startswith(tuple(f"{i}." for i in range(1, 13)))
        ]

        for line in numbered_lines:
            if ":" in line:
                key, value = line.split(":", 1)
                key = key.strip()
                value = value.strip()
                if key in combined_metadata:
                    combined_metadata[key] += f" | {value}"  # Concatenate values with a separator
                else:
                    combined_metadata[key] = value
    
    # Convert the combined metadata into a single-row DataFrame
    final_metadata = pd.DataFrame([combined_metadata])
    return final_metadata
        
    #     data = [
    #         line.split(":", 1) if ":" in line else (line, "")  # Handle cases with no details
    #         for line in numbered_lines
    #     ]
        
    #     # Convert the chunk's metadata to a DataFrame
    #     df_chunk = pd.DataFrame(data, columns=["Category", "Details"])
        
    #     # Ensure clean columns
    #     df_chunk["Category"] = df_chunk["Category"].str.strip()
    #     df_chunk["Details"] = df_chunk["Details"].str.strip()
        
    #     # Transpose to make categories columns
    #     df_transposed = pd.DataFrame([df_chunk["Details"].values], columns=df_chunk["Category"].values)
    #     df_transposed["Chunk_ID"] = i + 1  # Add Chunk ID for reference
        
    #     # Append to the list
    #     all_metadata.append(df_transposed)
    
    # # Combine all chunk results into a single DataFrame
    # final_metadata = pd.concat(all_metadata, ignore_index=True)
    # return final_metadata


In [9]:
# Define the models
MODEL_LLAMA_70B = "llama-3.3-70b-versatile"
MODEL_LLAMA_70B_specdec = "llama-3.3-70b-specdec"

# Extract metadata for all chunks using both models
print("Processing with MODEL_LLAMA_70B...")
llama_70b_responses = extract_metadata_from_chunks(client, MODEL_LLAMA_70B, cleaned_chunks)

print("Processing with MODEL_LLAMA_70B_specdec...")
llama_70b_spec_responses = extract_metadata_from_chunks(client, MODEL_LLAMA_70B_specdec, cleaned_chunks)

# Display responses
print("Llama 70B Metadata Results:")
print(llama_70b_responses)

print("Llama 70B Spec Metadata Results:")
print(llama_70b_spec_responses)

# Save results to CSV (optional)
llama_70b_responses.to_csv("llama_70b_metadata_results.csv", index=False)
llama_70b_spec_responses.to_csv("llama_70b_spec_metadata_results.csv", index=False)


Processing with MODEL_LLAMA_70B...
Processing chunk 1/49...
Processing chunk 2/49...
Processing chunk 3/49...
Processing chunk 4/49...
Processing chunk 5/49...
Processing chunk 6/49...
Processing chunk 7/49...
Processing chunk 8/49...
Processing chunk 9/49...
Processing chunk 10/49...
Processing chunk 11/49...
Processing chunk 12/49...
Processing chunk 13/49...
Processing chunk 14/49...
Processing chunk 15/49...
Processing chunk 16/49...
Processing chunk 17/49...
Processing chunk 18/49...
Processing chunk 19/49...
Processing chunk 20/49...
Processing chunk 21/49...
Processing chunk 22/49...
Processing chunk 23/49...
Processing chunk 24/49...
Processing chunk 25/49...
Processing chunk 26/49...
Processing chunk 27/49...
Processing chunk 28/49...
Processing chunk 29/49...
Processing chunk 30/49...
Processing chunk 31/49...
Processing chunk 32/49...
Processing chunk 33/49...
Processing chunk 34/49...
Processing chunk 35/49...
Processing chunk 36/49...
Processing chunk 37/49...
Processing c

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-specdec` in organization `org_01jgv12bvtexksmf5b91w65bfx` service tier `on_demand` on : Limit 100000, Used 100772, Requested 2673. Please try again in 49m37.143s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}

In [10]:
llama_70b_responses

,1. **NAME**,2. **JAHRGANG**,3. **ORT**,4. **GESCHLECHT**,5. **BERUF**,6. **VAT_JG**,7. **VAT_KONFESSION**,8. **VAT_HERKUN**,9. **VAT_SCHULE**,10. **VAT_AUSBIL**,...,3. ORT,4. GESCHLECHT,5. BERUF,6. VAT_JG,7. VAT_KONFESSION,8. VAT_HERKUN,9. VAT_SCHULE,10. VAT_AUSBIL,11. VAT_STAND,12. VAT_POLOR
0,Der Name der Person wird nicht explizit im Tra...,Der Jahrgang der Person wird nicht direkt gena...,Die Person ist in Altendorf geboren und hat do...,Das Geschlecht der Person wird nicht explizit ...,Die Person hat bei der Industrie- und Handelsk...,Das Geburtsjahr des Vaters wird nicht genannt.,Die Konfession des Vaters wird nicht erwähnt.,Die Herkunft des Vaters wird nicht spezifizier...,Die Schulbildung des Vaters wird nicht genannt.,Die Ausbildung des Vaters wird als Former besc...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Leider nicht explizit im Transkript erwähnt.,"1910 (berechnet aus dem Jahr 1933, in dem die ...","Der genaue Geburtsort wird nicht erwähnt, aber...",Männlich (impliziert durch die Erzählung und d...,"Zu Beginn des Transkripts wird erwähnt, dass d...",Nicht explizit erwähnt.,Evangelisch (da die Familie evangelisch war).,Ostpreußen.,Nicht erwähnt.,Nicht erwähnt.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Leider nicht explizit im Transkript erwähnt.,"Nicht direkt erwähnt, aber es wird von der Zei...","Der genaue Geburtsort wird nicht erwähnt, aber...","Männlich, basierend auf der Erzählung und der ...",Nicht explizit erwähnt.,Nicht erwähnt.,"Es wird erwähnt, dass der Vater aus einer länd...","Der Vater kam aus einer ländlichen Gegend, abe...",Nicht erwähnt.,Nicht erwähnt.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Leider nicht explizit im Transkript erwähnt.,"Der Sprecher erwähnt, dass er im Ersten Weltkr...","Der Sprecher erwähnt Orte wie Frintrop, Himmel...","Männlich, da der Sprecher sich selbst als ""Jun...","Der Sprecher erwähnt, dass er zur Firma Krupp ...",Nicht explizit erwähnt.,"Der Sprecher erwähnt, dass seine Eltern ""treu ...",Nicht erwähnt.,Nicht erwähnt.,Nicht erwähnt.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Der vollständige Name der Person wird nicht ex...,Das Geburtsjahr der Person wird nicht direkt g...,Der Geburtsort der Person wird nicht explizit ...,Die Person spricht über ihre Erfahrungen in de...,Der aktuelle Beruf der Person wird nicht im Tr...,Das Geburtsjahr des Vaters wird nicht genannt.,Die Religion des Vaters wird nicht explizit er...,Die Herkunft des Vaters wird nicht genannt.,Die Schulbildung des Vaters wird nicht erwähnt.,Die Ausbildung des Vaters wird nicht genannt.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Leider nicht explizit im Transkript erwähnt.,"Nicht direkt erwähnt, aber es wird erwähnt, da...","Der Sprecher erwähnt ""Altessen"" und ""Essen"" al...","Männlich, da der Sprecher sich selbst als ""ich...","Der Sprecher erwähnt, dass er bei der Handelsk...",Nicht erwähnt.,"Der Sprecher erwähnt ""Methodisten"" und ""evange...",Nicht erwähnt.,Nicht erwähnt.,Nicht erwähnt.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Leider nicht explizit im Transkript genannt.,Nicht genannt.,Nicht genannt.,"Männlich, basierend auf der Erzählung und der ...","Der Befragte war in der Verwaltung tätig, spez...",Nicht genannt.,Nicht genannt.,Nicht genannt.,Nicht genannt.,Nicht genannt.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Leider nicht explizit im Transkript erwähnt.,Nicht erwähnt.,Der Sprecher erwähnt Essen als seinen aktuelle...,"Männlich, basierend auf der Sprache und den Pr...","Der Sprecher erwähnt, dass er bei Krupp gearbe...",Nicht erwähnt.,Nicht erwähnt.,Nicht erwähnt.,Nicht erwähnt.,Nicht erwähnt.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Leider nicht explizit im Transkript erwähnt.,Nicht direkt im Transkript genannt.,"Der Sprecher erwähnt mehrere Orte, darunter Di...","Männlich, basierend auf der Erzählung und der ...",Der Sprecher war Verwaltungsmann bei Krupp.,Nicht im Transkript erwähnt.,Keine Information.,Keine Information.,Keine Information.,Keine Informati